<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir">بخش ۳) مدل تحلیل احساس
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این قسمت به حل مسئله‌ی خوش‌تعریف تحلیل احساس خواهید پرداخت. نیاز است مدلی طراحی کنید که با دریافت متن نظر کاربر، احساس/رضایت وی نسبت به کالا را بین عددی از ۱ تا ۵ تعیین کند. بنابراین متغیر هدف شما همان ستون overall خواهد بود. ورودی مدل شما می‌تواند علاوه بر متن نظر (reviewText) شامل خلاصه‌ی نظر یا اطلاعات دلخواه دیگری نیز باشد اما مبنای اصلی و الزامی کار همان متن نظر خواهد بود.
</font>
</p>

<div dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
برای این قسمت مجاز هستید از هر مدل دلخواهی استفاده کنید، اما به نکات زیر توجه کنید:

اگر از یک مدل پیش‌آموخته (pre-trained) استفاده می‌کنید، حتماً نیاز است آن را ویژه‌ی دامنه‌ی مسئله‌ی خود آموزش دهید (fine-tune کردن یا اضافه کردن لایه‌های دیگر).

تمام اعضای فعال گروه شما باید تسلط کافی نسبت به الگوریتم و پیاده‌سازی آن را داشته باشند. بنابراین اگر قصد استفاده از مدلی همچون ترنسفورمرها دارید سعی کنید معماری مورد استفاده را در گروه خود مطالعه و بررسی کنید.
</font>
</div>

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir">نکات کلی
</font>
</h3>

<div dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
لزومی به استفاده از تمامی داده‌های موجود در داده‌های آموزشی وجود ندارد. می‌توانید برای کاهش منابع سخت‌افزاری مورد نیاز برای فرآیند آموزش تنها از بخشی از مجموعه‌داده استفاده کنید.
فراموش نکنید که بخشی از داده‌های آموزشی را برای اعتبارسنجی (validation) جدا کنید.

استفاده از هر نوع پیش‌پردازش، کتابخانه و مدلی، آزاد است. تنها شرط لازم برای استفاده از موارد ذکر شده، تسلط تمامی اعضای فعال در گروه بر آن‌ها است.

بخش مهمی از این مسئله، نحوه‌ی پیش‌پردازش داده‌های متنی است. بنابراین سعی کنید از تکنیک‌های مختلفی جهت پیش‌پردازش هر چه بهتر متن‌ها بهره ببرید و نیاز است انتخاب‌های شما برای این مرحله همراه با دقت کافی و قابل استدلال باشد. به‌عنوان مثال اگر قصد حذف کلمات زائد (Stop words) را دارید دقت کنید که کلمات مهم برای این نوع مسئله‌ی خاص حذف نشوند
</font>
</div>

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix

### Preprocessing

In [2]:
data = pd.read_csv('../data/raw/train_data.csv', low_memory=False)
data.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600


In [3]:
X = data['reviewText']
y = data['overall']

In [4]:
# 50% of data for training, 30% for validation and 20% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.4, random_state=42)

In [6]:
# Convert to lowercase
X_train = X_train.apply(lambda x: x.lower())
X_val = X_val.apply(lambda x: x.lower())

In [7]:
# Remove extra spaces
X_train = X_train.apply(lambda x: ' '.join(x.split()))
X_val = X_val.apply(lambda x: ' '.join(x.split()))

In [12]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_val = tokenizer.texts_to_sequences(X_val)


In [13]:
X_train = pad_sequences(sequences_train, maxlen=128)
X_val = pad_sequences(sequences_val, maxlen=128)

In [14]:
embeddings_index = {}
with open('../data/external/glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Found {len(embeddings_index)} word vectors.')

Found 400000 word vectors.


In [15]:
embedding_dim = 100
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < len(tokenizer.word_index):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [22]:
y_train = y_train - 1
y_val = y_val - 1

### Modeling

In [18]:
model = Sequential()

# Embedding layer initialized with GloVe vectors
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    trainable=False))
# GRU layer
model.add(GRU(128, return_sequences=False)) 
# Dropout layer
model.add(Dropout(0.2))
# Output layer
model.add(Dense(5, activation='softmax'))

In [19]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [23]:
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val))


Epoch 1/5
6555/6555 ━━━━━━━━━━━━━━━━━━━━ 530s 81ms/step - accuracy: 0.6104 - loss: 1.0124 - val_accuracy: 0.6789 - val_loss: 0.8048
Epoch 2/5
6555/6555 ━━━━━━━━━━━━━━━━━━━━ 531s 81ms/step - accuracy: 0.6816 - loss: 0.7971 - val_accuracy: 0.6913 - val_loss: 0.7718
Epoch 3/5
6555/6555 ━━━━━━━━━━━━━━━━━━━━ 600s 91ms/step - accuracy: 0.7016 - loss: 0.7472 - val_accuracy: 0.7012 - val_loss: 0.7461
Epoch 4/5
6555/6555 ━━━━━━━━━━━━━━━━━━━━ 633s 97ms/step - accuracy: 0.7118 - loss: 0.7224 - val_accuracy: 0.7032 - val_loss: 0.7411
Epoch 5/5
6555/6555 ━━━━━━━━━━━━━━━━━━━━ 668s 102ms/step - accuracy: 0.7179 - loss: 0.7044 - val_accuracy: 0.7029 - val_loss: 0.7436


<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir">ارزیابی نهایی
</font>
</h3>

<div dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
مجموعه‌ی آزمونی که در اختیار شما قرار گرفته شامل برچسب حقیقی نیست. نیاز است پس از تکمیل کار خود، از مدل نهایی برای پیش‌بینی برچسب این نمونه‌ها استفاده کرده و یک فایل csv به شکل جدول زیر آماده کنید. پس از اتمام مهلت ارسال پروژه و آپلود فایل‌های شما، به مدت چند ساعت بخش جدیدی در سامانه باز خواهد شد تا بتوانید این فایل را آپلود کرده و نتیجه‌ی مدل خود را مشاهده کنید.

**معیار ارزیابی**: f1 score با روش میانگین‌گیری میکرو (micro)

**ساختار فایل**: نام فایل شما باید q2_submission.csv باشد و شامل یک ستون از احساس پیش‌بینی‌شده (predicted) باشد. ردیف اول باید مربوط به نمونه‌ی اول داده‌های آزمون،‌ ردیف دوم مربوط به نمونه‌ی دوم و الی آخر باشد. لطفاً نمایه‌ها (index) را نیز ذخیره نکنید.
</font>
</div>

### Evaluation

In [25]:
model.evaluate(X_val, y_val)

7866/7866 ━━━━━━━━━━━━━━━━━━━━ 178s 23ms/step - accuracy: 0.7039 - loss: 0.7427


[0.7436462044715881, 0.7029080390930176]

In [28]:
sequences_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences_test, maxlen=128)
y_test = y_test - 1

In [33]:
y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

5244/5244 ━━━━━━━━━━━━━━━━━━━━ 112s 21ms/step
              precision    recall  f1-score   support

           0       0.63      0.70      0.66     16537
           1       0.38      0.29      0.33     11342
           2       0.41      0.43      0.42     16265
           3       0.58      0.38      0.46     31400
           4       0.82      0.91      0.86     92245

    accuracy                           0.70    167789
   macro avg       0.56      0.55      0.55    167789
weighted avg       0.69      0.70      0.69    167789



In [35]:
model.save('model_glove.keras')